# Cookies & Code - Testing: What, Why, and How!

## Learning Intentions

- Explain when and why a piece of code needs tests.
-  Structure tests using the Arrange-Act-Assert framework.
-  Write and run simple tests using pytest \& ipytest.
-  Outline properties of testable code.
- Be equipped to start writing your own tests **today**.

# What is testing?


## What is pytest?
There are many packages that can be used to help test Python code.
`pytest` is known for its simple language and concise syntax, particularly compared to `unittest` which is part of the Python Standard Library.

You need to import `ipytest` to directly run pytest-style tests in a Jupyter Notebook.

In [ ]:
# Import at beginning of notebook
import ipytest
import pytest

ipytest.autoconfig()

You'll also notice commands beginning with `%%ipytest -qq` in cells containing tests.
We'll investigate their role soon!

## What is a test?
A test is a *function* that has an *assert statement*.

`pytest`, or `ipytest` runs the test functions. 

If its assert statements are:
- **true** — the test will **pass**!
- **false** or an **error** occurs — the test will **fail**.

Tests come in a few different flavours - we will deal with **unit tests** today. 

A unit test tests a small, isolated, piece of code (a unit).

For a piece of code to be testable, it needs to be:
- **Modular:** Able to be expressed in a function or class,
- **Deterministic:** The same inputs should always receive the same outputs,
- **De-coupled:** It should not interact with other components or systems.


## Example Set 1 

Try running (shift + enter) the following cells of examples.

In [ ]:
%%ipytest

def test_example():
    assert [1, 2, 3] == [1, 2, 3]

In [ ]:
%%ipytest
# What happens if you run the cell without the line above?

def test_will_fail():
    assert False

In [ ]:
%%ipytest

def this_test_will_not_run():
    # Pytest tests are required to begin with the test_ prefix.
    assert False

***Optional:** Experiment by replacing `%%ipytest` with:*
- `%%ipytest -vv`
- `%%ipytest -qq`

## Example Set 2 - Structuring Tests
Let's look at a slightly more interesting use case. We have a function "example_func" and we want to write some unit tests for it. 

We will write tests using the Arrange-Act-Assert Framework:

- **Arrange:** Create inputs to the function or class you are testing.
- **Act:** Call the function or class you are testing.
- **Assert:** Assert that you get the output you expected.

Look at how the tests below are structured:

In [ ]:
def example_func(x: int, y: int) -> int:
    return x + y

Try running the tests. Some should fail! Can you fix them?

In [ ]:
%%ipytest

def test_example_func():
    ## Arrange
    x = 10
    y = 15

    ## Act
    output = example_func(x,  y)

    ## Assert
    assert output == 25

def test_example_func_failing_test():
    ## This test fails! Can you fix it?
    ## NB: Requires changing 1 line.

    ## Arrange
    x = 20
    y = 15

    ## Act
    output = example_func(x, y)

    ## Assert
    assert output == 20

def test_example_func_fails_with_none():
    ## This test fails! What is the problem?
    ## NB: Requires changing 1 line.
    
    ## Arrange
    x = None
    y = 2

    ## Act & Assert
    with pytest.raises(ValueError):
        output = example_func(x, y)


# Why do we test?



- Writing automated unit tests allows us to change code and ensure we do not break existing functionality.
- Unit tests tell us what behaviour is expected from the code. They are a form of communication.

## Benefits of testing

- Less bugs! A test suite evolves as you fix bugs so they NEVER occur again.
- Documents the code.
- Forces you to write more modular code.

## Does this code need tests?

**Here is a thought experiment. Does this code need tests?:**

I have a piece of code that runs on a server every day at 9am.
	
It's run for 15 years and has not been altered since 1992.


## What code should I test?

Generally you should test code that meets one or more of the following conditions:
- Used in multiple places,
- Frequently changing or under active development,
- Important to be correct, or
- Complex, with lots of edge cases.

# How to Write Testable Code



As we said above, testable code should be:
- **Modular:** Able to be expressed in a function or class,
- **Deterministic:** The same inputs should always receive the same outputs,
- **De-coupled:** It should not interact with other components or systems.




## Example 3 - Refactoring

I have some code below that reads in some data from a file, and constructs some Star objects with some characteristics.

In [ ]:
import csv
from typing import List


class Star:
    def __init__(self, name: str, distance: float, luminosity: float):
        self.name = name
        self.distance = distance
        self.luminosity = luminosity


def process_astronomy_data(filename: str) -> List[Star]:
    """
    Reads a CSV file of star data and returns a list of Star objects.

    Args:
        filename (str): Path to the CSV file containing star data.

    Returns:
        List[Star]: A list of Star objects created from the CSV file.
    """
    stars = []
    with open(filename, newline="") as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            name = row["Name"]
            distance = float(row["Distance"])
            luminosity = float(row["Luminosity"])
            star = Star(name, distance, luminosity)
            stars.append(star)

    return stars

What makes testing the above function hard? Think about all the operations that are occurring.

In [ ]:
### Refactored version

from typing import List, Dict


def read_star_data(filename: str) -> List[Dict[str, str]]:
    """
    Reads a CSV file and returns the data as a list of dictionaries.

    Args:
        filename (str): Path to the CSV file containing star data.

    Returns:
        List[Dict[str, str]]: The raw data from the CSV as a list of dictionaries.
    """
    with open(filename, newline="") as csvfile:
        reader = csv.DictReader(csvfile, delimiter=",")
        return [row for row in reader]


def create_star_objects(data: List[Dict[str, str]]) -> List[Star]:
    """
    Converts raw star data into Star objects.

    Args:
        data (List[Dict[str, str]]): Raw data containing star information.

    Returns:
        List[Star]: A list of Star objects created from the raw data.
    """
    stars = []
    for row in data:
        name = row["Name"]
        distance = float(row["Distance"])
        luminosity = float(row["Luminosity"])
        star = Star(name, distance, luminosity)
        stars.append(star)
    return stars


def process_astronomy_data(filename: str) -> List[Star]:
    """
    Processes the astronomy data by reading and creating Star objects.

    Args:
        filename (str): Path to the CSV file containing star data.

    Returns:
        List[Star]: A list of Star objects.
    """
    raw_data = read_star_data(filename)
    return create_star_objects(raw_data)

In [ ]:
%%ipytest -qq
from unittest.mock import patch, mock_open

sample_data = [
    {"Name": "Star A", "Distance": "10", "Luminosity": "1000"},
    {"Name": "Star B", "Distance": "20", "Luminosity": "2000"}
]

@pytest.mark.parametrize("data,expected_stars", [
    (sample_data, [
        Star("Star A", 10.0, 1000.0),
        Star("Star B", 20.0, 2000.0)
    ])
])
def test_create_star_objects(data, expected_stars):
    stars = create_star_objects(data)
    for star, expected_star in zip(stars, expected_stars):
        assert star.name == expected_star.name
        assert star.distance == expected_star.distance
        assert star.luminosity == expected_star.luminosity

# Using a dummy file to test data ingestion. Look at fake_data.csv.
def test_process_astronomy_data():
    # This test fails. Can you use the debugger to figure out why?
    stars = process_astronomy_data("fake_data.csv")
    assert len(stars) == 2
    assert stars[0].name == "Star A"
    assert stars[1].name == "Star B"


    

# Parting Thoughts

How should you start testing in your work?

1. Start small. Pick a function or two that you use all the time, or a function you are about to change, and try to write some tests in a Jupyter notebook.
2. Don't test too much. Not every function requires a test.
3. When you find a bug in your code, write a test for it! Over time, your code will become bulletproof!

Reach out with any questions, I'm always happy to help.

# Fancier things - if you have time

## Run test .py scripts using Jupyter Notebooks

### Run a single file of tests

In [ ]:
!pytest test_example_functions.py -q

### Run all tests in directory

In [ ]:
!pytest

### Run tests with detailed outputs

In [ ]:
!pytest -vv

## Using pytest.mark.parameterize

Allows the same test function to run for many input / output pairs.

Note the number of tests that run when you run the below tests.

Try changing one of the input / output pairs to be incorrect. What happens?

In [ ]:
def square_number(x: int) -> int:
    return x**2

In [ ]:
%%ipytest -vv
import pytest

@pytest.mark.parametrize("input_num,expected_output", [
    (2, 4), (-2, 4), (8, 64)
])
def test_square_number(input_num, expected_output):
    ## Arrange
    ## Nothing to do here

    ## Act
    output = square_number(input_num)

    ## Assert
    assert output == expected_output

## Fixtures

Allow re-use of setup objects that you use again and again - for instance reading an input file. Here's an example from pycodif:

In [ ]:
## Just for an example - this won't run.
## The "example_frame" code is executed before each test
## and the output passed in the "example_frame" argument.
class TestCODIFFrame:

    @pytest.fixture()
    def example_frame(self):
        with open("tests/test_files/test_codif.codif", "rb") as f:
            codif = CODIFFrame(f)
        return codif

    def test_data_parsing(self, example_frame):
        assert hasattr(example_frame, "header")
        assert hasattr(example_frame, "data_array")
        assert hasattr(example_frame, "sample_timestamps")

    def test_data_values(self, example_frame):
        assert isinstance(example_frame.data_array, np.ndarray)
        assert example_frame.data_array.dtype == np.dtype("complex64")
        assert example_frame.data_array[0, 0] == -23 + 45j
        assert example_frame.data_array[0, -1] == 113 - 89j
        assert example_frame.data_array[-1, 0] == 45j
        assert example_frame.data_array[-1, -1] == -43 + 58j

## Hypothesis

This is a package that generates test cases for us based on properties.

In [ ]:
%%ipytest

# Taken from hypothesis quickstart guide:
# https://hypothesis.readthedocs.io/en/latest/quickstart.html
from hypothesis import given, strategies as st

@given(st.integers(0, 200))
def test_integers(n):
    assert n < 50

In [ ]:
%%ipytest
from typing import Union
import numpy as np

## Some of these fail. Is that expected?
## Can you fix them? 
## NB: Look at the section on filtering here: https://hypothesis.readthedocs.io/en/latest/quickstart.html

def square(x: Union[int, float]) -> Union[int, float]:
    return x ** 2

def square_root(x: Union[int, float]) -> Union[int, float]:
    return x ** 0.5

@given(s=st.integers())
def test_inverses_integers(s):
    assert np.isclose(s, square(square_root(s)))
    assert np.isclose(s, square_root(square(s)))

@given(s=st.floats())
def test_inverses_floats(s):
    assert np.isclose(s, square(square_root(s)))
    assert np.isclose(s, square_root(square(s)))

## Running tests on commit with GitHub Actions

You can also set up automatic jobs to run all tests when you commit your code to GitHub. This will send you an email if your tests fail!

Have a look at the file in the .github/workflows directory to see how this works. You may need to Show Hidden Files via Settings -> Settings Editor. Alternatively you can see it here: https://github.com/jdgsmallwood/cookies-testing/blob/main/.github/workflows/run-tests.yaml

You can also see the output of test runs at https://github.com/jdgsmallwood/cookies-testing/actions